**decorators: introduction**

Decorators are functions that make other functions better. It's a bit meta, but they're very powerful. In this series of video's we'll explore them by building our own but also by showing how you typically use them. There's no real code just yet. But this series of videos will go indepth so this might be a great moment to grab a coffee!

In python, functions can accept functions as input. This might seems a bit strange at first so let's look at an example.

In [1]:
def apply(func, a, b):
    return func(a, b)

def add(a, b):
    return a + b

def sub(a, b):
    return a - b

apply(add, 1, 2), apply(sub, 1, 2)

(3, -1)

But they can also return functions as output.

In [2]:
def power(n):
    def func(number):
        return number**n
    return func

pow2 = power(2)
pow3 = power(3)

pow2(3), pow3(3)


(9, 27)

 decorators: behavior

We've seen how functions can accept functions in python as input. Let's use this knowledge to create a function that accepts a function as input but also returns a function as output.

In [3]:
import time
import random

def stopwatch(f):
    def func():
        tic = time.time()
        result = f()
        print(f"this function took: {time.time() - tic}")
        return result
    return func

def sleep_random():
    time.sleep(random.random())
    return "Done!"

timed_sleep = stopwatch(sleep_random)


In [4]:
import time
import random

def stopwatch(f):
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

def sleep_random(s):
    t = s + random.random()
    time.sleep(t)
    return "Done"

timed_sleep = stopwatch(sleep_random)


You now have two functions that are similar, but also different.

In [5]:
sleep_random(s=2)
timed_sleep(s=2)


this function took: 2.028829336166382


'Done'

**decorators: decorate**

We've seen how functions can accept functions and output functions. Hopefully it is clear that we can use this technique to add behavior to functions.

For better syntax, let's use the @ symbol to add behavior to a function.

In [6]:
import time
import random

def stopwatch(f):
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

@stopwatch
def sleep_random(s):
    t = s + random.random()
    time.sleep(t)
    return f"Done"


Note that we now no longer have two functions, both one!

In [7]:
sleep_random(1)
sleep_random(2)
sleep_random(3)


this function took: 1.691352367401123
this function took: 2.3471806049346924
this function took: 3.976558208465576


'Done'

**decorators: wraps**

When we wrap a function with a decorator we might loose information from the original function. One of the things we miss out on is the docstring.

Let's make sure that the docstring is kept intact by using @wraps.

In [8]:
import time
import random

def stopwatch(f):
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

@stopwatch
def sleep_random(s):
    """This function sleeps at least for `s` seconds."""
    return time.sleep(s + random.random())

timed_sleep = stopwatch(sleep_random)


Note that we now no longer have two functions, but one!

In [9]:
help(sleep_random)


Help on function func in module __main__:

func(*args, **kwargs)



**decorators: stack**

You're not limited to a single decorator in python. You can apply many if you wish!

To ephesize that you can stack decorators, let's apply two new ones.

In [10]:
import time
import random
from functools import wraps

def print_call1(f):
    @wraps(f)
    def func(*args, **kwargs):
        print(f"print-call 1 args: {args}")
        result = f(*args, **kwargs)
        return result
    return func


def print_call2(f):
    @wraps(f)
    def func(*args, **kwargs):
        print(f"print-call 2 args: {args}")
        result = f(*args, **kwargs)
        return result
    return func

@print_call2
@print_call1
@print_call2
@print_call1
def sleep_random(s):
    """This function sleeps at least for `s` seconds."""
    return time.sleep(s + random.random()/100)

sleep_random(1.5)


print-call 2 args: (1.5,)
print-call 1 args: (1.5,)
print-call 2 args: (1.5,)
print-call 1 args: (1.5,)


**decorators: inputs**

We can also have decorators that accept inputs. This makes for even more expressiveness. You do need to pay attention when you're declaring them though.

The example below gives a full demonstration.

In [11]:
import time
import random
from functools import wraps

def loggg(show_name=True, show_time=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            log_text = "call"
            if show_name:
                log_text = f"{log_text} {f.__name__}"
            if show_time:
                log_text = f"{log_text} time:{time.time() - tic}"
            print(log_text)
            return result
        return func
    return stopwatch

@loggg(show_name=False, show_time=True)
def sleep_random(s):
    """This function sleeps at least for `s` seconds."""
    return time.sleep(s + random.random()/100)

sleep_random(1)


call time:1.0099332332611084


To make inputs optional, we need to do something nitty-gritty. We need to make sure that if no inputs are given, the decorated function is still able to run. The code below shows you how you might get that to work.

In [12]:
import time
import random
from functools import wraps

def loggg(func_in=None, *, show_name=True, show_time=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            result = "call"
            if show_name:
                result = f"{result} {f.__name__}"
            if show_time:
                result = f"{result} time:{time.time() - tic}"
            return result
        return func

    # This is where the "magic" happens.
    if func_in is None:
        return stopwatch
    else:
        return stopwatch(func_in)

@loggg
def sleep_random(s):
    """This function sleeps at least for `s` seconds."""
    return time.sleep(s + random.random()/100)

sleep_random(1)


'call sleep_random time:1.0027365684509277'

In [13]:
from functools import wraps
import datetime as dt

def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"just ran step {func.__name__} shape={result.shape} took {time_taken}s")
        return result
    return wrapper
